In [139]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.utils.data.dataloader as dl
import torchvision.transforms as transforms

In [140]:
class Net(nn.Module):
  def __init__(self, num_classes):
    super(Net, self).__init__()
    # instantiating 2 convos for MNIST data
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=5, out_channels=10, kernel_size=3, padding=1)
    self.relu = nn.ReLU()
    # instantiating maxpooling 
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    # Defining fully connected final layer with conv2 having 10 channels for 10 output classes:
    # Since the maxpooling(2, 2) will be applied two times, it divides the original heigh * width of MNIST images (28, 28) two times
    # So input will be of new dimension * channels 
    self.fc = nn.Linear(10 * 7 * 7, 10)

  def forward(self, x):
    # Apply conv followd by relu, then pool
    x = self.relu(self.conv1(x))
    x = self.pool(x)
    x = self.relu(self.conv2(x))
    x = self.pool(x)
    # Prepare the image for the fully connected layer
    x = x.view(-1, 7 * 7 * 10)

    # Applying the fully connected layer and return the result
    return self.fc(x)

In [141]:
# building the model with loss and optimizers
net = Net(10)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)

In [142]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307), (0.3081))])
trainset = torchvision.datasets.MNIST(root='./sample_data/mnist_data', train=True, transform=transform, download=True)
testset = torchvision.datasets.MNIST(root='./sample_data/mnist_data', train=False, transform=transform, download=True)
trainloader = dl.DataLoader(trainset, shuffle=True, batch_size=5, num_workers=4)
testloader = dl.DataLoader(testset, shuffle=False, batch_size=5, num_workers=4)
display(trainloader.dataset.data.shape)

torch.Size([60000, 28, 28])

In [143]:
# Using single epoch training the CNN Model
for i, data in enumerate(trainloader, 0):
  inputs, labels = data
  # clear the previous gradients
  optimizer.zero_grad()
  # forward pass:
  outputs = net(inputs)

  # compute loss:
  loss = criterion(outputs, labels)
  # calculate gradient
  loss.backward()
  # update weights:
  optimizer.step()

print("Training finished")

Training finished


In [144]:
# Evaluating the CNN model:
net.eval()
predictions = []
total, correct = 0, 0
for i, data in enumerate(testloader):
  inputs, labels = data
  outputs = net(inputs)
  _, predicted = torch.max(outputs, 1)
  predictions.append(outputs)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()


In [145]:
print("The accuracy on test set for CNN Model is:", correct/total * 100)

The accuracy on test set for CNN Model is: 97.49
